In [1]:
import math

def fixed_point_method(initial_guess, tolerance=1e-4, max_iterations=1000):
    x_prev = initial_guess
    iteration = 0

    while iteration < max_iterations:
        x_next = math.exp(-x_prev)
        
        # Check for convergence
        if abs(x_next - x_prev) < tolerance:
            return round(x_next, 4)
        
        x_prev = x_next
        iteration += 1

    raise ValueError("Fixed-point method did not converge within the specified number of iterations.")

# Initial guess
initial_guess = 0.0

# Solve the equation
result = fixed_point_method(initial_guess)

print("Solution:", result)


Solution: 0.5671


In [10]:
import numpy as np

# Define the integrand
def integrand(x):
    return 1 / np.sqrt(1 + x**4)

# Apply Simpson's rule
def simpsons_rule(func, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = func(x)
    result = h / 3 * (y[0] + 4 * np.sum(y[1:-1:2]) + 2 * np.sum(y[2:-2:2]) + y[-1])
    return result

# Apply Gaussian quadrature (using 3-point rule)
def gaussian_quadrature(func, a, b):
    # Weights and nodes for 3-point Gaussian quadrature
    weights = np.array([5/9, 8/9, 5/9])
    nodes = np.array([-np.sqrt(3/5), 0, np.sqrt(3/5)])

    # Change of interval from [-1, 1] to [a, b]
    x = 0.5 * (b - a) * nodes + 0.5 * (a + b)
    w = 0.5 * (b - a) * weights

    # Evaluate the integrand and sum up
    result = np.sum(w * func(x))
    return result

# Interval for the integral
a, b = 0, 1

# Number of subintervals for Simpson's rule
n_simpson = 1000  # You can adjust this for desired accuracy

# Apply Simpson's rule
simpson_result = simpsons_rule(integrand, a, b, n_simpson)

# Apply Gaussian quadrature
gaussian_result = gaussian_quadrature(integrand, a, b)

print("Result using Simpson's rule:", round(simpson_result, 6))
print("Result using Gaussian quadrature:", round(gaussian_result, 6))


Result using Simpson's rule: 0.927037
Result using Gaussian quadrature: 0.927184


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the ODE
def ode(x, y):
    return 5 * x**2 - y * np.exp(x + y)

# RK4 method for solving ODE
def runge_kutta_4th_order(x, y, h):
    k1 = h * ode(x, y)
    k2 = h * ode(x + 0.5 * h, y + 0.5 * k1)
    k3 = h * ode(x + 0.5 * h, y + 0.5 * k2)
    k4 = h * ode(x + h, y + k3)
    return y + (k1 + 2*k2 + 2*k3 + k4) / 6

# Initial conditions
x0, y0 = 0, 1.0

# Interval sizes
interval_sizes = [0.5, 0.2, 0.05, 0.01]

# Tabulate results
for h in interval_sizes:
    x_values = np.arange(x0, 1.0 + h, h)
    y_values = [y0]

    for i in range(1, len(x_values)):
        y_next = runge_kutta_4th_order(x_values[i-1], y_values[i-1], h)
        y_values.append(y_next)

    # Print the results
    print(f"\nInterval size: {h}")
    print("x \t\t y")
    for x_val, y_val in zip(x_values, y_values):
        print(f"{x_val:.2f} \t {y_val:.6f}")

    # Plot the results
    plt.plot(x_values, y_values, label=f'h = {h}')

# Show the plot
plt.xlabel('x')
plt.ylabel('y')
plt.title('RK4 Method for ODE')
plt.legend()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def crank_nicolson_solver(L, T, Nx, Nt, alpha):
    # Discretization
    dx = L / (Nx - 1)
    dt = T / Nt

    x_values = np.linspace(0, L, Nx)
    t_values = np.linspace(0, T, Nt)

    # Initial condition
    u_initial = 4 * x_values - x_values**2 / 2

    # Initialize solution matrix
    u = np.zeros((Nx, Nt))

    # Set initial condition
    u[:, 0] = u_initial

    # Crank-Nicolson method
    A = np.eye(Nx) * (2 + 4 * alpha) - np.eye(Nx, k=-1) * alpha - np.eye(Nx, k=1) * alpha
    B = np.eye(Nx) * (2 - 4 * alpha) + np.eye(Nx, k=-1) * alpha + np.eye(Nx, k=1) * alpha

    for n in range(1, Nt):
        b = B @ u[:, n-1]
        u[:, n] = np.linalg.solve(A, b)

    return x_values, t_values, u

# Parameters
L = 8.0  # Length of the rod
T = 1.0  # Total time
Nx = 100  # Number of spatial points
Nt = 100  # Number of time points
alpha = 0.01  # Stability parameter

# Solve using Crank-Nicolson method
x_values, t_values, u = crank_nicolson_solver(L, T, Nx, Nt, alpha)

# Display the solution in a table
print("x \t\t t \t\t u")
for i, xi in enumerate(x_values):
    for j, tj in enumerate(t_values):
        print(f"{xi:.4f} \t {tj:.4f} \t {u[i, j]:.6f}")

# Display the solution in a contour plot
plt.contourf(t_values, x_values, u.T, cmap='viridis')
plt.colorbar(label='Temperature')
plt.xlabel('Time')
plt.ylabel('Position')
plt.title('Crank-Nicolson Solution to Heat Equation')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def poisson_solver(nx, ny):
    # Define the grid
    x = np.linspace(0, 2, nx)
    y = np.linspace(0, 1, ny)

    # Set boundary conditions
    u = np.zeros((nx, ny))
    u[:, 0] = x
    u[:, -1] = x * np.exp(1)
    u[0, :] = 0
    u[-1, :] = 2 * np.exp(y)

    # Solve the Poisson's equation using finite differences
    for _ in range(1000):  # Iterative solver, adjust as needed
        u_new = u.copy()

        for i in range(1, nx - 1):
            for j in range(1, ny - 1):
                u_new[i, j] = (u[i + 1, j] + u[i - 1, j] + u[i, j + 1] + u[i, j - 1] - x[i] * np.exp(y[j])) / 4

        u = u_new

    return x, y, u

# Grid size
nx = 62
ny = 62

# Solve Poisson's equation
x, y, solution = poisson_solver(nx, ny)

# Display the solution in a table
print("x \t\t y \t\t u")
for i, xi in enumerate(x):
    for j, yj in enumerate(y):
        print(f"{xi:.4f} \t {yj:.4f} \t {solution[i, j]:.6f}")

# Display the solution in a 3-D plot
X, Y = np.meshgrid(x, y)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, solution.T, cmap='viridis')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('U')
ax.set_title('Poisson\'s Equation Solution')
plt.show()
